# Option Chain Analysis Notebook

This notebook provides comprehensive analysis of option chains data by joining underlying securities, option contracts, and market data prices.

## Features:
- Complete option chain data with latest prices
- Option chain summaries and aggregations
- Arbitrage opportunity scanning
- Data visualizations and analysis
- Interactive filtering and exploration

**Author**: AlcLite Trading System  
**Created**: 2025-08-08

## 1. Setup and Database Connection

In [2]:
# Import required libraries
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

# Set matplotlib style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Libraries imported successfully")

✅ Libraries imported successfully


In [16]:
# Database configuration
DB_CONFIG = {
    'host': 'localhost',
    'port': 5433,
    'database': 'options_arbitrage',
    'user': 'trading_user',
    'password': 'secure_trading_password'
}

# Create database connection
def get_db_connection():
    """Create and return a database connection."""
    try:
        connection_string = f"postgresql://{DB_CONFIG['user']}:{DB_CONFIG['password']}@{DB_CONFIG['host']}:{DB_CONFIG['port']}/{DB_CONFIG['database']}"
        engine = create_engine(connection_string)
        return engine
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return None

# Test connection
engine = get_db_connection()
if engine:
    print("✅ Database connection established")
    
    # Test with a simple query
    test_query = "SELECT COUNT(*) as total_underlyings FROM underlying_securities WHERE active = true"
    result = pd.read_sql(test_query, engine)
    print(f"📊 Found {result['total_underlyings'].iloc[0]} active underlying securities")
    query = "SELECT * FROM underlying_securities WHERE active = true"
    result = pd.read_sql(query, engine)

    display(result)


else:
    print("❌ Failed to establish database connection")

✅ Database connection established
📊 Found 6 active underlying securities


,id,symbol,name,sector,industry,market_cap,active,created_at,updated_at
0,2,AAPL,None,None,None,None,True,2025-08-04 18:22:32.095854+00:00,2025-08-04 18:22:32.095854+00:00
1,5,IWM,None,None,None,None,True,2025-08-04 18:23:43.327152+00:00,2025-08-04 18:23:43.327152+00:00
2,6,IBM,None,None,None,None,True,2025-08-04 19:54:15.349601+00:00,2025-08-05 04:03:55.973932+00:00
3,1,SPY,None,None,None,None,True,2025-08-04 18:22:32.079197+00:00,2025-08-05 06:30:13.944341+00:00
4,24,QQQ,None,None,None,None,True,2025-08-05 07:10:43.952730+00:00,2025-08-05 07:10:59.858678+00:00
5,3,TSLA,None,None,None,None,True,2025-08-04 18:22:32.097234+00:00,2025-08-05 07:48:36.327680+00:00


In [4]:
# Helper functions

def run_query(query, params=None, description=""):
    """Execute a SQL query and return results as DataFrame."""
    try:
        start_time = datetime.now()
        
        if params:
            df = pd.read_sql(query, engine, params=params)
        else:
            df = pd.read_sql(query, engine)
            
        execution_time = (datetime.now() - start_time).total_seconds()
        
        print(f"✅ Query executed successfully in {execution_time:.2f}s")
        if description:
            print(f"📝 {description}")
        print(f"📊 Returned {len(df)} rows, {len(df.columns)} columns")
        
        return df
        
    except Exception as e:
        print(f"❌ Query execution failed: {e}")
        return pd.DataFrame()

def format_currency(value):
    """Format numeric value as currency."""
    if pd.isna(value):
        return "N/A"
    return f"${value:,.2f}"

def format_percentage(value):
    """Format numeric value as percentage."""
    if pd.isna(value):
        return "N/A"
    return f"{value:.2%}"

print("✅ Helper functions defined")

✅ Helper functions defined


## 2. Query 1: Complete Option Chain with Latest Prices

This query provides a comprehensive view of all option chains with their most recent market data, including:
- Underlying security information
- Current stock price
- Option contract details (strikes, expirations, types)
- Latest bid/ask prices and volumes
- Greeks (delta, gamma, theta, vega, rho)
- Implied volatility
- Moneyness calculations

In [9]:
# Complete Option Chain Query
complete_option_chain_query = """
WITH latest_option_prices AS (
    -- Get the most recent price data for each option contract
    SELECT DISTINCT ON (contract_id)
        contract_id,
        time,
        bid_price,
        ask_price,
        last_price,
        bid_size,
        ask_size,
        volume,
        open_interest,
        delta,
        gamma,
        theta,
        vega,
        rho,
        implied_volatility,
        bid_ask_spread,
        mid_price,
        tick_type
    FROM market_data_ticks
    ORDER BY contract_id, time DESC
),
latest_stock_prices AS (
    -- Get the most recent stock price for each underlying
    SELECT DISTINCT ON (underlying_id)
        underlying_id,
        time as stock_time,
        price as stock_price,
        volume as stock_volume
    FROM stock_data_ticks
    ORDER BY underlying_id, time DESC
)
SELECT 
    -- Underlying information
    us.symbol,
    us.name,
    us.sector,
    us.industry,
    
    -- Current stock price
    sp.stock_price,
    sp.stock_volume,
    sp.stock_time,
    
    -- Option contract details
    oc.expiration_date,
    oc.strike_price,
    oc.option_type,
    oc.contract_symbol,
    oc.multiplier,
    
    -- Days to expiration
    (oc.expiration_date - CURRENT_DATE) as days_to_expiry,
    
    -- Moneyness calculations
    CASE 
        WHEN oc.option_type = 'C' THEN 
            CASE 
                WHEN sp.stock_price > oc.strike_price THEN 'ITM'
                WHEN sp.stock_price = oc.strike_price THEN 'ATM'
                ELSE 'OTM'
            END
        ELSE -- Put option
            CASE 
                WHEN sp.stock_price < oc.strike_price THEN 'ITM'
                WHEN sp.stock_price = oc.strike_price THEN 'ATM'
                ELSE 'OTM'
            END
    END as moneyness,
    
    -- Distance from ATM as percentage
    ROUND(((oc.strike_price / sp.stock_price) - 1) * 100, 2) as strike_premium_pct,
    
    -- Price data
    op.bid_price,
    op.ask_price,
    op.last_price,
    op.mid_price,
    op.bid_ask_spread,
    op.bid_size,
    op.ask_size,
    op.volume as option_volume,
    op.open_interest,
    
    -- Greeks
    op.delta,
    op.gamma,
    op.theta,
    op.vega,
    op.rho,
    op.implied_volatility,
    
    -- Timestamps
    op.time as option_time,
    op.tick_type
    
FROM underlying_securities us
JOIN option_chains oc ON us.id = oc.underlying_id
LEFT JOIN latest_option_prices op ON oc.id = op.contract_id
LEFT JOIN latest_stock_prices sp ON us.id = sp.underlying_id
WHERE us.active = true 
  AND oc.active = true
  AND oc.expiration_date > CURRENT_DATE
ORDER BY us.symbol, oc.expiration_date, oc.strike_price, oc.option_type;
"""

# Execute the query
option_chain_df = run_query(
    complete_option_chain_query, 
    description="Complete option chain with latest prices and Greeks"
)

print(f"\n📈 Option Chain Data Summary:")
print(f"Symbols: {option_chain_df['symbol'].nunique() if not option_chain_df.empty else 0}")
print(f"Total Option Contracts: {len(option_chain_df)}")
print(f"Expirations: {option_chain_df['expiration_date'].nunique() if not option_chain_df.empty else 0}")

# Display first few rows
if not option_chain_df.empty:
    display(option_chain_df.head(100))

✅ Query executed successfully in 0.01s
📝 Complete option chain with latest prices and Greeks
📊 Returned 56 rows, 32 columns

📈 Option Chain Data Summary:
Symbols: 2
Total Option Contracts: 56
Expirations: 2


,symbol,name,sector,industry,stock_price,stock_volume,stock_time,expiration_date,strike_price,option_type,contract_symbol,multiplier,days_to_expiry,moneyness,strike_premium_pct,bid_price,ask_price,last_price,mid_price,bid_ask_spread,bid_size,ask_size,option_volume,open_interest,delta,gamma,theta,vega,rho,implied_volatility,option_time,tick_type
0,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,205.0,C,IBM781714663,100,7,ITM,-18.65,46.3815,47.3185,46.85,46.85,0.9370,100,100,-1,None,None,None,None,None,None,None,2025-08-04 19:59:00+00:00,HISTORICAL
1,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,205.0,P,IBM781716450,100,7,OTM,-18.65,0.0594,0.0606,0.06,0.06,0.0012,100,100,-1,None,None,None,None,None,None,None,2025-07-24 19:59:00+00:00,HISTORICAL
2,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,210.0,C,IBM781714707,100,7,ITM,-16.67,41.4810,42.3190,41.90,41.90,0.8380,100,100,-1,None,None,None,None,None,None,None,2025-08-04 19:59:00+00:00,HISTORICAL
3,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,210.0,P,IBM781716518,100,7,OTM,-16.67,0.5247,0.5353,0.53,0.53,0.0106,100,100,-1,None,None,None,None,None,None,None,2025-08-01 19:59:00+00:00,HISTORICAL
4,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,215.0,C,IBM781714735,100,7,ITM,-14.69,36.1152,36.8448,36.48,36.48,0.7296,100,100,-1,None,None,None,None,None,None,None,2025-08-04 19:59:00+00:00,HISTORICAL
5,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,215.0,P,IBM781716554,100,7,OTM,-14.69,0.1782,0.1818,0.18,0.18,0.0036,100,100,-1,None,None,None,None,None,None,None,2025-07-29 16:59:00+00:00,HISTORICAL
6,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,220.0,C,IBM781714775,100,7,ITM,-12.70,31.6305,32.2695,31.95,31.95,0.6390,100,100,-1,None,None,None,None,None,None,None,2025-08-04 19:59:00+00:00,HISTORICAL
7,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,220.0,P,IBM781716617,100,7,OTM,-12.70,0.1683,0.1717,0.17,0.17,0.0034,100,100,-1,None,None,None,None,None,None,None,2025-08-04 19:59:00+00:00,HISTORICAL
8,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,225.0,C,IBM781714815,100,7,ITM,-10.72,26.9577,27.5023,27.23,27.23,0.5446,100,100,-1,None,None,None,None,None,None,None,2025-08-04 19:59:00+00:00,HISTORICAL
9,IBM,None,None,None,252.01,888,2025-08-04 19:59:00+00:00,2025-08-15,225.0,P,IBM781716666,100,7,OTM,-10.72,0.2376,0.2424,0.24,0.24,0.0048,100,100,-1,None,None,None,None,None,None,None,2025-08-04 19:59:00+00:00,HISTORICAL


## 3. Query 2: Option Chain Summary by Strike

This query provides a side-by-side comparison of calls and puts for each strike price, making it easy to analyze option chains in a traditional format.

In [6]:
# Option Chain Summary Query
option_summary_query = """
WITH latest_prices AS (
    SELECT DISTINCT ON (contract_id)
        contract_id,
        bid_price,
        ask_price,
        last_price,
        volume,
        open_interest,
        implied_volatility,
        delta,
        time
    FROM market_data_ticks
    ORDER BY contract_id, time DESC
),
latest_stock_prices AS (
    SELECT DISTINCT ON (underlying_id)
        underlying_id,
        price as stock_price
    FROM stock_data_ticks
    ORDER BY underlying_id, time DESC
)
SELECT 
    us.symbol,
    sp.stock_price,
    oc.expiration_date,
    oc.strike_price,
    (oc.expiration_date - CURRENT_DATE) as days_to_expiry,
    
    -- Call option data
    MAX(CASE WHEN oc.option_type = 'C' THEN op.bid_price END) as call_bid,
    MAX(CASE WHEN oc.option_type = 'C' THEN op.ask_price END) as call_ask,
    MAX(CASE WHEN oc.option_type = 'C' THEN op.last_price END) as call_last,
    MAX(CASE WHEN oc.option_type = 'C' THEN op.volume END) as call_volume,
    MAX(CASE WHEN oc.option_type = 'C' THEN op.open_interest END) as call_oi,
    MAX(CASE WHEN oc.option_type = 'C' THEN op.implied_volatility END) as call_iv,
    MAX(CASE WHEN oc.option_type = 'C' THEN op.delta END) as call_delta,
    
    -- Put option data  
    MAX(CASE WHEN oc.option_type = 'P' THEN op.bid_price END) as put_bid,
    MAX(CASE WHEN oc.option_type = 'P' THEN op.ask_price END) as put_ask,
    MAX(CASE WHEN oc.option_type = 'P' THEN op.last_price END) as put_last,
    MAX(CASE WHEN oc.option_type = 'P' THEN op.volume END) as put_volume,
    MAX(CASE WHEN oc.option_type = 'P' THEN op.open_interest END) as put_oi,
    MAX(CASE WHEN oc.option_type = 'P' THEN op.implied_volatility END) as put_iv,
    MAX(CASE WHEN oc.option_type = 'P' THEN op.delta END) as put_delta,
    
    -- Calculations
    ROUND(((oc.strike_price / sp.stock_price) - 1) * 100, 2) as strike_premium_pct,
    
    -- Synthetic stock price from put-call parity
    ROUND(
        MAX(CASE WHEN oc.option_type = 'C' THEN op.last_price END) - 
        MAX(CASE WHEN oc.option_type = 'P' THEN op.last_price END) + 
        oc.strike_price, 2
    ) as synthetic_stock_price
    
FROM underlying_securities us
JOIN option_chains oc ON us.id = oc.underlying_id
LEFT JOIN latest_prices op ON oc.id = op.contract_id
LEFT JOIN latest_stock_prices sp ON us.id = sp.underlying_id
WHERE us.active = true 
  AND oc.active = true
  AND oc.expiration_date > CURRENT_DATE
GROUP BY us.symbol, sp.stock_price, oc.expiration_date, oc.strike_price
HAVING MAX(CASE WHEN oc.option_type = 'C' THEN op.last_price END) IS NOT NULL
    OR MAX(CASE WHEN oc.option_type = 'P' THEN op.last_price END) IS NOT NULL
ORDER BY us.symbol, oc.expiration_date, oc.strike_price;
"""

# Execute the query
option_summary_df = run_query(
    option_summary_query,
    description="Option chain summary with calls and puts side by side"
)

print(f"\n📊 Option Summary Data:")
print(f"Strike/Expiry combinations: {len(option_summary_df)}")

# Display first few rows
if not option_summary_df.empty:
    display(option_summary_df.head(10))

✅ Query executed successfully in 0.00s
📝 Option chain summary with calls and puts side by side
📊 Returned 28 rows, 21 columns

📊 Option Summary Data:
Strike/Expiry combinations: 28


,symbol,stock_price,expiration_date,strike_price,days_to_expiry,call_bid,call_ask,call_last,call_volume,call_oi,call_iv,call_delta,put_bid,put_ask,put_last,put_volume,put_oi,put_iv,put_delta,strike_premium_pct,synthetic_stock_price
0,IBM,252.01,2025-08-15,205.0,7,46.3815,47.3185,46.85,-1,None,None,None,0.0594,0.0606,0.06,-1,None,None,None,-18.65,251.79
1,IBM,252.01,2025-08-15,210.0,7,41.4810,42.3190,41.90,-1,None,None,None,0.5247,0.5353,0.53,-1,None,None,None,-16.67,251.37
2,IBM,252.01,2025-08-15,215.0,7,36.1152,36.8448,36.48,-1,None,None,None,0.1782,0.1818,0.18,-1,None,None,None,-14.69,251.30
3,IBM,252.01,2025-08-15,220.0,7,31.6305,32.2695,31.95,-1,None,None,None,0.1683,0.1717,0.17,-1,None,None,None,-12.70,251.78
4,IBM,252.01,2025-08-15,225.0,7,26.9577,27.5023,27.23,-1,None,None,None,0.2376,0.2424,0.24,-1,None,None,None,-10.72,251.99
5,IBM,252.01,2025-08-15,227.5,7,23.9382,24.4218,24.18,-1,None,None,None,0.2673,0.2727,0.27,-1,None,None,None,-9.73,251.41
6,IBM,252.01,2025-08-15,230.0,7,21.9780,22.4220,22.20,-1,None,None,None,0.2277,0.2323,0.23,-1,None,None,None,-8.73,251.97
7,IBM,252.01,2025-08-15,232.5,7,20.0673,20.4727,20.27,-1,None,None,None,0.3861,0.3939,0.39,-1,None,None,None,-7.74,252.38
8,IBM,252.01,2025-08-15,235.0,7,16.9092,17.2508,17.08,-1,None,None,None,0.4059,0.4141,0.41,-1,None,None,None,-6.75,251.67
9,IBM,252.01,2025-08-22,205.0,14,46.1835,47.1165,46.65,-1,None,None,None,0.3861,0.3939,0.39,-1,None,None,None,-18.65,251.26


## 4. Query 3: Arbitrage Scanner

This query identifies potential arbitrage opportunities by comparing actual stock prices with synthetic stock prices derived from option pricing.

In [7]:
# Arbitrage Scanner Query
arbitrage_scanner_query = """
WITH current_prices AS (
    SELECT DISTINCT ON (contract_id)
        contract_id,
        bid_price,
        ask_price,
        mid_price,
        volume,
        open_interest,
        time
    FROM market_data_ticks
    WHERE tick_type IN ('REALTIME', 'DELAYED', 'HISTORICAL')
      AND bid_price > 0 AND ask_price > 0
    ORDER BY contract_id, time DESC
),
current_stock_prices AS (
    SELECT DISTINCT ON (underlying_id)
        underlying_id,
        price as stock_price,
        time as stock_time
    FROM stock_data_ticks
    ORDER BY underlying_id, time DESC
),
option_pairs AS (
    SELECT 
        us.symbol,
        us.id as underlying_id,
        csp.stock_price,
        csp.stock_time,
        c_oc.expiration_date,
        c_oc.strike_price,
        (c_oc.expiration_date - CURRENT_DATE) as days_to_expiry,
        
        -- Call data
        c_prices.bid_price as call_bid,
        c_prices.ask_price as call_ask,
        c_prices.mid_price as call_mid,
        c_prices.volume as call_volume,
        c_prices.open_interest as call_oi,
        
        -- Put data
        p_prices.bid_price as put_bid,
        p_prices.ask_price as put_ask,
        p_prices.mid_price as put_mid,
        p_prices.volume as put_volume,
        p_prices.open_interest as put_oi,
        
        -- Synthetic calculations
        (c_prices.bid_price - p_prices.ask_price + c_oc.strike_price) as synthetic_stock_bid,
        (c_prices.ask_price - p_prices.bid_price + c_oc.strike_price) as synthetic_stock_ask,
        (c_prices.mid_price - p_prices.mid_price + c_oc.strike_price) as synthetic_stock_mid
        
    FROM underlying_securities us
    JOIN current_stock_prices csp ON us.id = csp.underlying_id
    JOIN option_chains c_oc ON (us.id = c_oc.underlying_id AND c_oc.option_type = 'C')
    JOIN option_chains p_oc ON (us.id = p_oc.underlying_id 
        AND p_oc.expiration_date = c_oc.expiration_date 
        AND p_oc.strike_price = c_oc.strike_price 
        AND p_oc.option_type = 'P')
    LEFT JOIN current_prices c_prices ON c_oc.id = c_prices.contract_id
    LEFT JOIN current_prices p_prices ON p_oc.id = p_prices.contract_id
    WHERE us.active = true 
      AND c_oc.active = true 
      AND p_oc.active = true
      AND c_oc.expiration_date > CURRENT_DATE
      AND c_oc.expiration_date <= CURRENT_DATE + INTERVAL '120 days'  -- Focus on near-term options
      AND c_prices.bid_price IS NOT NULL 
      AND p_prices.ask_price IS NOT NULL
      AND c_prices.volume > 0 AND p_prices.volume > 0  -- Ensure liquidity
)
SELECT 
    symbol,
    stock_price,
    stock_time,
    expiration_date,
    strike_price,
    days_to_expiry,
    
    -- Option prices
    call_bid,
    call_ask,
    put_bid,
    put_ask,
    call_volume,
    put_volume,
    call_oi,
    put_oi,
    
    -- Synthetic stock prices
    ROUND(synthetic_stock_bid, 4) as synthetic_stock_bid,
    ROUND(synthetic_stock_ask, 4) as synthetic_stock_ask,
    ROUND(synthetic_stock_mid, 4) as synthetic_stock_mid,
    
    -- Arbitrage calculations
    ROUND(stock_price - synthetic_stock_ask, 4) as buy_synthetic_profit,  -- Buy synthetic, sell stock
    ROUND(synthetic_stock_bid - stock_price, 4) as sell_synthetic_profit, -- Buy stock, sell synthetic
    
    -- Maximum available profit
    ROUND(GREATEST(
        stock_price - synthetic_stock_ask,     -- Buy synthetic, sell stock
        synthetic_stock_bid - stock_price      -- Buy stock, sell synthetic
    ), 4) as max_profit,
    
    -- Profit as percentage of strike
    ROUND(GREATEST(
        stock_price - synthetic_stock_ask,
        synthetic_stock_bid - stock_price
    ) / strike_price * 100, 3) as profit_pct,
    
    -- Synthetic vs actual spread
    ROUND(ABS(synthetic_stock_mid - stock_price), 4) as price_discrepancy
    
FROM option_pairs
WHERE (
    -- Look for significant arbitrage opportunities
    ABS(stock_price - synthetic_stock_ask) > 0.05 OR
    ABS(synthetic_stock_bid - stock_price) > 0.05
)
ORDER BY 
    GREATEST(
        stock_price - synthetic_stock_ask,
        synthetic_stock_bid - stock_price
    ) DESC,
    days_to_expiry ASC;
"""

# Execute the query
arbitrage_df = run_query(
    arbitrage_scanner_query,
    description="Arbitrage opportunities scanner"
)

print(f"\n🔍 Arbitrage Scanner Results:")
print(f"Potential opportunities: {len(arbitrage_df)}")

if not arbitrage_df.empty:
    print(f"Max profit opportunity: ${arbitrage_df['max_profit'].max():.4f}")
    print(f"Average profit: ${arbitrage_df['max_profit'].mean():.4f}")
    display(arbitrage_df.head(15))
else:
    print("No arbitrage opportunities found with current criteria")

✅ Query executed successfully in 0.01s
📝 Arbitrage opportunities scanner
📊 Returned 0 rows, 22 columns

🔍 Arbitrage Scanner Results:
Potential opportunities: 0
No arbitrage opportunities found with current criteria


## 5. Data Analysis and Filtering

Let's create some interactive analysis tools to explore the data.

In [8]:
# Function to analyze a specific symbol
def analyze_symbol(symbol, max_days_to_expiry=60):
    """Analyze option chain for a specific symbol."""
    
    if option_chain_df.empty:
        print("❌ No option chain data available")
        return
    
    # Filter data for the symbol
    symbol_data = option_chain_df[
        (option_chain_df['symbol'] == symbol) & 
        (option_chain_df['days_to_expiry'] <= max_days_to_expiry)
    ].copy()
    
    if symbol_data.empty:
        print(f"❌ No data found for {symbol}")
        return
    
    print(f"\n📊 Analysis for {symbol}")
    print(f"Current Stock Price: ${symbol_data['stock_price'].iloc[0]:.2f}")
    print(f"Total Option Contracts: {len(symbol_data)}")
    print(f"Expirations: {symbol_data['expiration_date'].nunique()}")
    print(f"Strike Range: ${symbol_data['strike_price'].min():.0f} - ${symbol_data['strike_price'].max():.0f}")
    
    # Summary by expiration
    exp_summary = symbol_data.groupby('expiration_date').agg({
        'strike_price': ['count', 'min', 'max'],
        'volume': 'sum',
        'open_interest': 'sum',
        'days_to_expiry': 'first'
    }).round(2)
    
    exp_summary.columns = ['Contracts', 'Min Strike', 'Max Strike', 'Total Volume', 'Total OI', 'DTE']
    
    print("\n📅 By Expiration:")
    display(exp_summary)
    
    return symbol_data

# Example: Analyze SPY if available
if not option_chain_df.empty:
    available_symbols = option_chain_df['symbol'].unique()
    print(f"Available symbols: {', '.join(available_symbols[:10])}...")
    
    # Analyze the first symbol or SPY if available
    target_symbol = 'SPY' if 'SPY' in available_symbols else available_symbols[0]
    spy_data = analyze_symbol(target_symbol)

Available symbols: IBM, TSLA...

📊 Analysis for IBM
Current Stock Price: $252.01
Total Option Contracts: 36
Expirations: 2
Strike Range: $205 - $235


KeyError: "Column(s) ['volume'] do not exist"

## 6. Data Visualizations

In [ ]:
def plot_option_chain(symbol_data, expiration_date=None):
    """Plot option chain visualization."""
    
    if symbol_data is None or symbol_data.empty:
        print("❌ No data to plot")
        return
    
    # Filter by expiration if specified
    if expiration_date:
        data = symbol_data[symbol_data['expiration_date'] == expiration_date].copy()
    else:
        # Use the nearest expiration
        nearest_exp = symbol_data['expiration_date'].min()
        data = symbol_data[symbol_data['expiration_date'] == nearest_exp].copy()
        expiration_date = nearest_exp
    
    if data.empty:
        print(f"❌ No data for expiration {expiration_date}")
        return
    
    symbol = data['symbol'].iloc[0]
    stock_price = data['stock_price'].iloc[0]
    
    # Separate calls and puts
    calls = data[data['option_type'] == 'C'].copy()
    puts = data[data['option_type'] == 'P'].copy()
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'{symbol} Option Chain - {expiration_date} (Stock: ${stock_price:.2f})', fontsize=16)
    
    # 1. Volatility Smile
    ax1 = axes[0, 0]
    if not calls.empty and calls['implied_volatility'].notna().any():
        ax1.plot(calls['strike_price'], calls['implied_volatility'], 'bo-', label='Calls', alpha=0.7)
    if not puts.empty and puts['implied_volatility'].notna().any():
        ax1.plot(puts['strike_price'], puts['implied_volatility'], 'ro-', label='Puts', alpha=0.7)
    ax1.axvline(stock_price, color='green', linestyle='--', alpha=0.7, label='Stock Price')
    ax1.set_xlabel('Strike Price')
    ax1.set_ylabel('Implied Volatility')
    ax1.set_title('Volatility Smile')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Option Prices
    ax2 = axes[0, 1]
    if not calls.empty and calls['last_price'].notna().any():
        ax2.plot(calls['strike_price'], calls['last_price'], 'bo-', label='Call Prices', alpha=0.7)
    if not puts.empty and puts['last_price'].notna().any():
        ax2.plot(puts['strike_price'], puts['last_price'], 'ro-', label='Put Prices', alpha=0.7)
    ax2.axvline(stock_price, color='green', linestyle='--', alpha=0.7, label='Stock Price')
    ax2.set_xlabel('Strike Price')
    ax2.set_ylabel('Option Price ($)')
    ax2.set_title('Option Prices')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Volume Analysis
    ax3 = axes[1, 0]
    strikes = sorted(set(calls['strike_price']).union(set(puts['strike_price'])))
    call_vols = [calls[calls['strike_price'] == s]['volume'].sum() for s in strikes]
    put_vols = [puts[puts['strike_price'] == s]['volume'].sum() for s in strikes]
    
    x = np.arange(len(strikes))
    width = 0.35
    ax3.bar(x - width/2, call_vols, width, label='Call Volume', alpha=0.7, color='blue')
    ax3.bar(x + width/2, put_vols, width, label='Put Volume', alpha=0.7, color='red')
    ax3.set_xlabel('Strike Price')
    ax3.set_ylabel('Volume')
    ax3.set_title('Trading Volume by Strike')
    ax3.set_xticks(x[::max(1, len(strikes)//10)])  # Show every nth strike
    ax3.set_xticklabels([f'{s:.0f}' for s in strikes[::max(1, len(strikes)//10)]])
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. Open Interest
    ax4 = axes[1, 1]
    call_oi = [calls[calls['strike_price'] == s]['open_interest'].sum() for s in strikes]
    put_oi = [puts[puts['strike_price'] == s]['open_interest'].sum() for s in strikes]
    
    ax4.bar(x - width/2, call_oi, width, label='Call OI', alpha=0.7, color='blue')
    ax4.bar(x + width/2, put_oi, width, label='Put OI', alpha=0.7, color='red')
    ax4.set_xlabel('Strike Price')
    ax4.set_ylabel('Open Interest')
    ax4.set_title('Open Interest by Strike')
    ax4.set_xticks(x[::max(1, len(strikes)//10)])
    ax4.set_xticklabels([f'{s:.0f}' for s in strikes[::max(1, len(strikes)//10)]])
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Plot option chain for the analyzed symbol
if 'spy_data' in locals() and spy_data is not None:
    plot_option_chain(spy_data)

## 7. Arbitrage Opportunities Visualization

In [ ]:
def plot_arbitrage_opportunities(arb_data, top_n=20):
    """Visualize arbitrage opportunities."""
    
    if arb_data is None or arb_data.empty:
        print("❌ No arbitrage data to plot")
        return
    
    # Get top opportunities
    top_arb = arb_data.head(top_n).copy()
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Arbitrage Opportunities Analysis', fontsize=16)
    
    # 1. Profit Distribution
    ax1 = axes[0, 0]
    ax1.hist(arb_data['max_profit'], bins=30, alpha=0.7, color='green', edgecolor='black')
    ax1.set_xlabel('Max Profit ($)')
    ax1.set_ylabel('Frequency')
    ax1.set_title('Profit Distribution')
    ax1.grid(True, alpha=0.3)
    
    # 2. Profit by Days to Expiry
    ax2 = axes[0, 1]
    scatter = ax2.scatter(arb_data['days_to_expiry'], arb_data['max_profit'], 
                         c=arb_data['max_profit'], cmap='RdYlGn', alpha=0.7)
    ax2.set_xlabel('Days to Expiry')
    ax2.set_ylabel('Max Profit ($)')
    ax2.set_title('Profit vs Days to Expiry')
    ax2.grid(True, alpha=0.3)
    plt.colorbar(scatter, ax=ax2)
    
    # 3. Top Opportunities by Symbol
    ax3 = axes[1, 0]
    symbol_profits = arb_data.groupby('symbol')['max_profit'].sum().sort_values(ascending=True).tail(10)
    symbol_profits.plot(kind='barh', ax=ax3, color='skyblue')
    ax3.set_xlabel('Total Profit Opportunity ($)')
    ax3.set_title('Top Symbols by Total Profit')
    ax3.grid(True, alpha=0.3)
    
    # 4. Price Discrepancy Analysis
    ax4 = axes[1, 1]
    ax4.scatter(arb_data['price_discrepancy'], arb_data['max_profit'], alpha=0.6)
    ax4.set_xlabel('Price Discrepancy ($)')
    ax4.set_ylabel('Max Profit ($)')
    ax4.set_title('Profit vs Price Discrepancy')
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"\n📈 Arbitrage Summary:")
    print(f"Total opportunities: {len(arb_data)}")
    print(f"Average profit: ${arb_data['max_profit'].mean():.4f}")
    print(f"Max profit: ${arb_data['max_profit'].max():.4f}")
    print(f"Total potential profit: ${arb_data['max_profit'].sum():.2f}")
    
    # Top opportunities table
    print(f"\n🏆 Top {min(10, len(arb_data))} Opportunities:")
    top_cols = ['symbol', 'strike_price', 'expiration_date', 'days_to_expiry', 
               'stock_price', 'max_profit', 'profit_pct']
    display(arb_data[top_cols].head(10))

# Plot arbitrage opportunities if available
if not arbitrage_df.empty:
    plot_arbitrage_opportunities(arbitrage_df)
else:
    print("No arbitrage opportunities to visualize")

## 8. Export and Save Functions

In [ ]:
# Export functions
def export_data_to_csv(df, filename_prefix):
    """Export DataFrame to CSV with timestamp."""
    if df.empty:
        print(f"❌ No data to export for {filename_prefix}")
        return
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{filename_prefix}_{timestamp}.csv"
    
    try:
        df.to_csv(filename, index=False)
        print(f"✅ Data exported to {filename} ({len(df)} rows)")
    except Exception as e:
        print(f"❌ Export failed: {e}")

def export_all_data():
    """Export all analysis results."""
    print("📤 Exporting analysis results...")
    
    export_data_to_csv(option_chain_df, "option_chain_complete")
    export_data_to_csv(option_summary_df, "option_chain_summary")
    export_data_to_csv(arbitrage_df, "arbitrage_opportunities")
    
    print("\n✅ All exports completed!")

# Uncomment to export data
# export_all_data()

## 9. Interactive Analysis Tools

Use these cells to perform custom analysis:

In [ ]:
# Custom Analysis Cell
# Modify this cell to run custom queries or analysis

custom_query = """
-- Example: Find options with highest implied volatility
SELECT 
    us.symbol,
    oc.expiration_date,
    oc.strike_price,
    oc.option_type,
    mdt.implied_volatility,
    mdt.volume,
    mdt.open_interest
FROM underlying_securities us
JOIN option_chains oc ON us.id = oc.underlying_id
JOIN (
    SELECT DISTINCT ON (contract_id)
        contract_id, implied_volatility, volume, open_interest
    FROM market_data_ticks
    WHERE implied_volatility IS NOT NULL
    ORDER BY contract_id, time DESC
) mdt ON oc.id = mdt.contract_id
WHERE us.active = true 
  AND oc.active = true
  AND mdt.implied_volatility > 0
ORDER BY mdt.implied_volatility DESC
LIMIT 20;
"""

# Execute custom query
custom_results = run_query(custom_query, description="Custom analysis - Highest IV options")

if not custom_results.empty:
    display(custom_results)

In [ ]:
# Quick Symbol Analysis
# Change the symbol here to analyze different stocks

ANALYZE_SYMBOL = "SPY"  # Change this to any symbol you want to analyze

if not option_chain_df.empty and ANALYZE_SYMBOL in option_chain_df['symbol'].values:
    print(f"\n🔍 Quick Analysis for {ANALYZE_SYMBOL}")
    
    symbol_data = analyze_symbol(ANALYZE_SYMBOL, max_days_to_expiry=90)
    
    if symbol_data is not None and not symbol_data.empty:
        # Show ATM options
        stock_price = symbol_data['stock_price'].iloc[0]
        atm_options = symbol_data[
            abs(symbol_data['strike_price'] - stock_price) <= 5  # Within $5 of stock price
        ].copy()
        
        if not atm_options.empty:
            print(f"\n📍 Near ATM Options (Stock: ${stock_price:.2f}):")
            atm_display = atm_options[[
                'expiration_date', 'strike_price', 'option_type', 'moneyness',
                'bid_price', 'ask_price', 'last_price', 'implied_volatility', 'volume'
            ]].sort_values(['expiration_date', 'strike_price', 'option_type'])
            display(atm_display.head(15))
else:
    print(f"❌ Symbol {ANALYZE_SYMBOL} not found in the data")
    if not option_chain_df.empty:
        print(f"Available symbols: {', '.join(option_chain_df['symbol'].unique()[:10])}")

## 10. Summary and Notes

This notebook provides comprehensive analysis of option chain data by joining:
- `underlying_securities` - Stock information
- `option_chains` - Option contract details
- `market_data_ticks` - Current option prices and Greeks
- `stock_data_ticks` - Current stock prices

### Key Queries:
1. **Complete Option Chain**: All option data with latest prices and Greeks
2. **Option Summary**: Side-by-side calls and puts by strike
3. **Arbitrage Scanner**: Identifies pricing discrepancies and opportunities

### Usage Tips:
- Modify the `ANALYZE_SYMBOL` variable to analyze different stocks
- Adjust the `max_days_to_expiry` parameter to focus on different time horizons
- Use the custom query cell for specialized analysis
- Export results using the export functions

### Performance Notes:
- Queries use `DISTINCT ON` for efficient latest price retrieval
- Filters limit results to active securities and unexpired options
- Consider adding date filters for very large datasets